# Using GraphRAG-SDK to Create a Multi-Agent-Orchestrator RAG System based on Knowledge Graph from Structured Data

GraphRAG-SDK provides a powerful tool, enhanced by LLM technology, for building a Multi-Agent-Orchestrator RAG System. This example demonstrates how to create a knowledge graph-based multi-agent system from structured data and how to use the orchestrator to manage the agents and perform question answering.

In [ ]:
!pip install graphrag_sdk

In [1]:

from json import loads
from dotenv import load_dotenv
from graphrag_sdk.orchestrator import Orchestrator
from graphrag_sdk.agents.kg_agent import KGAgent
from graphrag_sdk.models.openai import OpenAiGenerativeModel
from graphrag_sdk import (
    Ontology, Entity, Relation, Attribute, AttributeType, KnowledgeGraph, KnowledgeGraphModelConfig
)

# Load environment variables
load_dotenv()

# Configuration
# OPENAI_API_KEY = "sk-"# OpenAI API key

True

### Ontology - Predefined Ontology

We will begin with an initial ontology for the agents, based on the knowledge that interests us.


In [3]:
# Restaurants Ontology
restaurants_ontology = Ontology()

# Attractions Ontology
attractions_ontology = Ontology()

# Manually created Ontology by adding entities and relations
restaurants_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_entity(
    Entity(
        label="Restaurant",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="rating",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
            Attribute(
                name="food_type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
restaurants_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Restaurant",
        target="City",
    )
)

attractions_ontology.add_entity(
    Entity(
        label="Country",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="City",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="weather",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="population",
                attr_type=AttributeType.NUMBER,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_entity(
    Entity(
        label="Attraction",
        attributes=[
            Attribute(
                name="name",
                attr_type=AttributeType.STRING,
                required=True,
                unique=True,
            ),
            Attribute(
                name="description",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
            Attribute(
                name="type",
                attr_type=AttributeType.STRING,
                required=False,
                unique=False,
            ),
        ],
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_COUNTRY",
        source="City",
        target="Country",
    )
)
attractions_ontology.add_relation(
    Relation(
        label="IN_CITY",
        source="Attraction",
        target="City",
    )
)

### KG from Structure Data

Creating the KG from the data that will be used by the agents

In [4]:
# Define the model
model = OpenAiGenerativeModel("gpt-4o")

# Create the KG from the predefined ontology.
# Restaurants KG
restaurants_kg = KnowledgeGraph(
    name="restaurants",
    ontology=restaurants_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Attractions KG
attractions_kg = KnowledgeGraph(
    name="attractions",
    ontology=attractions_ontology,
    model_config=KnowledgeGraphModelConfig.with_model(model),
)

# Add nodes and edges to the KGs from the data
with open("examples/trip/data/cities.json") as f:
    cities = loads(f.read())
with open("examples/trip/data/restaurants.json") as f:
    restaurants = loads(f.read())
with open("examples/trip/data/attractions.json") as f:
    attractions = loads(f.read())

for city in cities:
    restaurants_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    restaurants_kg.add_node("Country", {"name": city["country"]})
    restaurants_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

    attractions_kg.add_node(
        "City",
        {
            "name": city["name"],
            "weather": city["weather"],
            "population": city["population"],
        },
    )
    attractions_kg.add_node("Country", {"name": city["country"]})
    attractions_kg.add_edge(
        "IN_COUNTRY",
        "City",
        "Country",
        {"name": city["name"]},
        {"name": city["country"]},
    )

for restaurant in restaurants:
    restaurants_kg.add_node(
        "Restaurant",
        {
            "name": restaurant["name"],
            "description": restaurant["description"],
            "rating": restaurant["rating"],
            "food_type": restaurant["food_type"],
        },
    )
    restaurants_kg.add_edge(
        "IN_CITY",
        "Restaurant",
        "City",
        {"name": restaurant["name"]},
        {"name": restaurant["city"]},
    )

for attraction in attractions:
    attractions_kg.add_node(
        "Attraction",
        {
            "name": attraction["name"],
            "description": attraction["description"],
            "type": attraction["type"],
        },
    )
    attractions_kg.add_edge(
        "IN_CITY",
        "Attraction",
        "City",
        {"name": attraction["name"]},
        {"name": attraction["city"]},
    )

### Agents

Creating the Agents based on the KG.

In [5]:
# The following agent is specialized in finding restaurants.
restaurants_agent = KGAgent(
    agent_id="restaurants_agent",
    kg=restaurants_kg,
    introduction="I'm a restaurant agent, specialized in finding the best restaurants for you.",
)

# The following agent is specialized in finding tourist attractions.
attractions_agent = KGAgent(
    agent_id="attractions_agent",
    kg=attractions_kg,
    introduction="I'm an attractions agent, specialized in finding the best attractions for you.",
)

### Orchestrator - Multi-Agent System

The orchestrator manages the usage of agents and handles questioning. Each agent is registered with the orchestrator, which determines how to engage the agents to provide the best possible answer when a question is asked.

In [6]:

# Initialize the orchestrator and register agents
orchestrator = Orchestrator(
    model,
    backstory="You are a trip planner, and you want to provide the best possible itinerary for your clients.",
)
orchestrator.register_agent(restaurants_agent)
orchestrator.register_agent(attractions_agent)

# Query the orchestrator
runner = orchestrator.ask("Create a two-day itinerary for a trip to Rome. Please don't ask me any questions; just provide the best itinerary you can.")

In [7]:
print(runner.output)

Certainly! Here is a well-rounded two-day itinerary for your trip to Rome, designed to help you explore the city's rich history, stunning architecture, and delightful culinary offerings.

**Day 1: Historical Treasures and Local Cuisine**
- **Morning:**
  - Start your day at the **Colosseum**, a must-see landmark symbolizing Rome's imperial power and infamous gladiatorial contests. Take a guided tour to fully appreciate the historical significance of this ancient amphitheater.
- **Afternoon:**
  - Head over to the **Basilica di Santa Maria Maggiore**, one of Rome's major basilicas, renowned for its remarkable architecture and long-standing history. Explore its beautiful mosaics and extensive collection of Roman artifacts.
- **Evening:**
  - For dinner, visit **Trattoria da Enzo**, a cozy and welcoming trattoria celebrated for its traditional Roman dishes. With a 4.5 rating, it offers a perfect atmosphere to unwind and savor classic Italian flavors.

**Day 2: Art and Gourmet Experiences*

In [8]:
runner = orchestrator.ask("Please tell me only the name of the restaurant for dinner at the second day that you mention in the itinerary of the trip")
print(runner.output)

Certainly! Based on the detailed itinerary I previously provided for your two-day trip to Rome, the restaurant mentioned for dinner on the second day is **Il Pagliaccio**. It is a Michelin-starred restaurant praised for its contemporary Italian cuisine, and it promises a memorable dining experience. Enjoy your culinary adventure at this elegant establishment!


In [9]:
runner = orchestrator.ask("Can you change this restaurant to another one and give me the updated itinerary?")
print(runner.output)

I appreciate your interest in customizing the itinerary for your trip to Rome. However, upon querying the restaurant agent for an alternative dinner option on the second day, it appears that there are currently no alternative restaurant recommendations available for that part of your itinerary.

Nonetheless, the current itinerary recommends **Il Pagliaccio**, a renowned Michelin-starred restaurant known for its exceptional contemporary Italian cuisine. It remains an exquisite choice for a memorable dining experience during your visit.

If you would like to explore more options, I encourage you to check local guides or recent reviews during your stay, as new dining venues and experiences frequently emerge in dynamic cities like Rome. Thank you for understanding, and I hope you have a fantastic trip filled with delightful experiences.


In [10]:
runner = orchestrator.ask("Please tell me only the name of the restaurant for dinner at the first day that you mention in the itinerary of the trip")
print(runner.output)

Certainly! From the two-day itinerary I provided for your trip to Rome, the restaurant mentioned for dinner on the first day is **Trattoria da Enzo**. This cozy trattoria is celebrated for its traditional Roman dishes and welcoming atmosphere, making it a wonderful choice for unwinding and enjoying classic Italian flavors. Enjoy your dining experience at this charming spot!
